In [1]:
import sys 
sys.path.append('../src') 
import numpy as np
#for some reason, matplotlib crashes without these lines
a = np.zeros((5,5))
a@a
import matplotlib.pyplot as plt
from utils import get_mesh_vtk
from utils import get_data
from utils import strip_cross, reconstruct_cross
from KPCA import *
import KPCA
import Autoencoder
# from torchsummary import summary
# from Autoencoder import Autoencoder
import torch
import torch.nn as nn
from utils import calc_energy, torch2np
import glob
import os
# from importlib import reload

In [2]:
def get_t(filename):
    str_s = filename.split('_')[4].replace('t','')
    t = float(str_s)
    return t

def read_from_ds(Wi, beta, ds_path=None):
    if ds_path is None:
        ds_path = '../../npz_data/four_roll_train'

    files = glob.glob(f'*Wi{Wi:g}_beta{beta:g}_*.pt', root_dir=ds_path)
    X = torch.load(f'{ds_path}/{files[0]}')['tensor']
    P = torch.load(f'{ds_path}/{files[0]}')['param']
    data = torch.zeros([len(files)] + [s for s in X.shape])
    params = torch.zeros([len(files)] + [len(P)])


    files = np.array(files)
    times = [get_t(fname) for fname in files]
    idx = np.argsort(times)
    files = files[idx]
    for i,file in enumerate(files):
        data_file = torch.load(f'{ds_path}/{file}')
        data[i] = data_file['tensor'].float()
        params[i] = torch.tensor(data_file['param']).float()
    return data, params

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [21]:
Re = 1.0
parameters = [[2.0, 0.11111],
 [2.0, 0.4],
 [2.0, 0.75],
 [3.0, 0.11111],
 [3.0, 0.4],
 [3.0, 0.75],
 [3.5, 0.11111],
 [3.5, 0.4],
 [3.5, 0.75],
 [3.8, 0.11111],
 [3.8, 0.4],
 [3.8, 0.75]]

In [15]:
dspath = '../../npz_data/four_roll_train_osc'
X_list = []
param_list = []
for p in parameters:
    X_torch, param = read_from_ds(*p, ds_path=dspath)

    X_list.append(torch2np(X_torch))
    param_list.append(param[:,:2].numpy())
    

In [19]:
Xpca = np.hstack(X_list)
param_kpca = np.vstack(param_list)

In [22]:
Wi_arr = param_kpca[:,0]
beta_arr = param_kpca[:,1]

theta_pca = ((1-beta_arr)/(Re * Wi_arr))[:,None]
theta_pca_sqrt = np.sqrt(theta_pca@theta_pca.T)

print('Theta Shape',theta_pca.shape)
print('Theta train Shape',theta_pca_sqrt.shape)

Theta Shape (36000, 1)
Theta train Shape (36000, 36000)


In [ ]:
KPCA_linear = KPCA.KernelPCA()
KPCA_oldroyd = KPCA.KernelPCA()

In [ ]:
degree = 1
ncomp = 3 
dx = 1/(2**6)
KPCA_linear = KPCA.KernelPCA()
KPCA_oldroyd = KPCA.KernelPCA()

KPCA_linear.fit(Xpca.T, n_components=ncomp, kernel='linear', theta=theta_pca_sqrt, eps = None, degree = degree, use_chol = 200)
KPCA_oldroyd.fit(Xpca.T, n_components=ncomp, kernel='oldroyd', theta=theta_pca_sqrt, eps = None, dx = dx, dy = dx, degree = degree, use_chol = 200)